## 1. Install

In [ ]:
!pip install pydub speechrecognition
!apt-get install ffmpeg -y


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 14.0 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


In [ ]:
!pip install openai-whisper


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 11.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.7 MB/s

## 2.Google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## 3. Whisper used with  mp3


In [ ]:
import os
from pydub import AudioSegment
import whisper

def mp3_to_wav(input_path):
    """Converts MP3 file to WAV format and returns the path of the temporary WAV file."""
    if not os.path.exists(input_path):
        raise FileNotFoundError(f"The file {input_path} does not exist.")

    temp_wav = 'temp_audio.wav'
    audio_segment = AudioSegment.from_mp3(input_path)
    audio_segment.export(temp_wav, format='wav')
    return temp_wav

def transcribe_audio(audio_file):
    """Transcribes audio from the given file path using Whisper."""
    try:
        model = whisper.load_model("base")  # You can use "small", "medium", or "large" for better accuracy
        result = model.transcribe(audio_file)
        return result["text"]
    except Exception as e:
        print(f"Error during transcription: {e}")
        return None

def run_mp3_to_txt(input_path):
    # input_path = input("Enter the path to your MP3 file: ")

    try:
        # Convert MP3 to WAV
        temp_wav = mp3_to_wav(input_path)

        # Transcribe audio using Whisper
        text = transcribe_audio(temp_wav)

        if text is not None:
            output_path = os.path.splitext(input_path)[0] + '_transcription.txt'
            with open(output_path, 'w', encoding='utf-8') as f:
                f.write(text)
            print(f"Transcription saved to {output_path}")
        else:
            print("No transcription was generated. Please check the audio file.")

    except Exception as e:
        print(f"An error occurred: {e}")


In [ ]:
# run_mp3_to_txt()

In [ ]:
#@title Transcribe mp3 to txt with whisper { display-mode: "form" }
# Data = "data_dst" #@param ["data_src", "data_dst"]
input_path = "/content/drive/MyDrive/mp3_to_md_whisper/2025-03-20-11-54-0ProteinEngineeringAIandResponsibleInnovation.mp3" #@param [""] {allow-input: true}

# print(fade_param)
# # fade_param

run_mp3_to_txt(input_path)

100%|████████████████████████████████████████| 139M/139M [00:00<00:00, 210MiB/s]


Transcription saved to /content/drive/MyDrive/mp3_to_md_whisper/2025-03-20-11-54-0ProteinEngineeringAIandResponsibleInnovation_transcription.txt


## 3 Reading txt file and translating from any language any language (50 possible)

In [ ]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
import jieba
import chardet
import os
from tqdm import tqdm  # Import tqdm for progress bar
import torch  # PyTorch for GPU support

def translate_text(file_path, source_lang='zh_CN', target_lang='en_XX', max_length=512):
    # Check if GPU is available and set device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")  # Show if using GPU or CPU

    # Load model and tokenizer
    model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt").to(device)
    tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

    try:
        # Detect encoding
        with open(file_path, 'rb') as file:
            raw_data = file.read()
            result = chardet.detect(raw_data)
            encoding = result['encoding'] if result['confidence'] > 0.8 else 'utf-8'

        # Read the text from the file
        with open(file_path, 'r', encoding=encoding) as file:
            text = file.read()

        # Apply segmentation using jieba for better tokenization
        segmented_text = " ".join(jieba.cut(text))

        # Split text into manageable chunks if too long
        if len(segmented_text) > max_length:
            print("⚠️ Text too long; splitting into smaller chunks for translation.")
            text_chunks = [segmented_text[i:i + max_length] for i in range(0, len(segmented_text), max_length)]
        else:
            text_chunks = [segmented_text]

        # Create the output file path with language-specific prefix
        directory, filename = os.path.split(file_path)
        new_filename = f"_{source_lang}_{target_lang}_{filename}"
        output_path = os.path.join(directory, new_filename)

        # Initialize list to store translations
        translations = []

        # Use tqdm to show progress as we process the chunks
        print(f"Processing {len(text_chunks)} chunks...")

        # Iterate over chunks with tqdm for a progress bar
        for chunk in tqdm(text_chunks, desc="Translating chunks", unit="chunk"):
            tokenizer.src_lang = source_lang
            encoded_text = tokenizer(chunk, return_tensors="pt", truncation=True, padding=True).to(device)
            generated_tokens = model.generate(
                **encoded_text,
                forced_bos_token_id=tokenizer.lang_code_to_id[target_lang],
                max_length=512
            )
            translation = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
            translations.append(translation)

        # Save the translation to the output file
        with open(output_path, 'w', encoding='utf-8') as output_file:
            output_file.write(" ".join(translations))

        return f"Translation saved to {output_path}"

    except FileNotFoundError:
        return "File not found."
    except Exception as e:
        return f"An error occurred: {e}"

# # Example usage
# result = translate_text('/content/drive/MyDrive/mp3_to_md_whisper/biomap-2025-03-18-08-23-15_transcription.txt')
# print(result)


# Any to any translation

In [ ]:
# Arabic (ar_AR), Czech (cs_CZ), German (de_DE), English (en_XX), Spanish (es_XX), Estonian (et_EE), Finnish (fi_FI), French (fr_XX), Gujarati (gu_IN), Hindi (hi_IN), Italian (it_IT), Japanese (ja_XX), Kazakh (kk_KZ), Korean (ko_KR), Lithuanian (lt_LT), Latvian (lv_LV), Burmese (my_MM), Nepali (ne_NP), Dutch (nl_XX), Romanian (ro_RO), Russian (ru_RU), Sinhala (si_LK), Turkish (tr_TR), Vietnamese (vi_VN), Chinese (zh_CN), Afrikaans (af_ZA), Azerbaijani (az_AZ), Bengali (bn_IN), Persian (fa_IR), Hebrew (he_IL), Croatian (hr_HR), Indonesian (id_ID), Georgian (ka_GE), Khmer (km_KH), Macedonian (mk_MK), Malayalam (ml_IN), Mongolian (mn_MN), Marathi (mr_IN), Polish (pl_PL), Pashto (ps_AF), Portuguese (pt_XX), Swedish (sv_SE), Swahili (sw_KE), Tamil (ta_IN), Telugu (te_IN), Thai (th_TH), Tagalog (tl_XX), Ukrainian (uk_UA), Urdu (ur_PK), Xhosa (xh_ZA), Galician (gl_ES), Slovene (sl_SI)


In [ ]:
#@title reading txt fiel and translating from  any language t other language{ display-mode: "form" }
# Data = "data_dst" #@param ["data_src", "data_dst"]
input_txt_path = "/content/drive/MyDrive/mp3_to_md_whisper/fr2" #@param [""] {allow-input: true}
source_lang = "fr_XX" # @param ["ar_AR", "cs_CZ", "de_DE", "en_XX", "es_XX", "et_EE", "fi_FI", "fr_XX", "gu_IN", "hi_IN", "it_IT", "ja_XX", "kk_KZ", "ko_KR", "lt_LT", "lv_LV", "my_MM", "ne_NP", "nl_XX", "ro_RO", "ru_RU", "si_LK", "tr_TR", "vi_VN", "zh_CN", "af_ZA", "az_AZ", "bn_IN", "fa_IR", "he_IL", "hr_HR", "id_ID", "ka_GE", "km_KH", "mk_MK", "ml_IN", "mn_MN", "mr_IN", "pl_PL", "ps_AF", "pt_XX", "sv_SE", "sw_KE", "ta_IN", "te_IN", "th_TH", "tl_XX", "uk_UA", "ur_PK", "xh_ZA", "gl_ES", "sl_SI"]
# {"allow-input":true}
# target_lang = "pl_PL" # @param ["pl_PL","en_XX"] {"allow-input":true}
target_lang = "pl_PL" # @param ["ar_AR", "cs_CZ", "de_DE", "en_XX", "es_XX", "et_EE", "fi_FI", "fr_XX", "gu_IN", "hi_IN", "it_IT", "ja_XX", "kk_KZ", "ko_KR", "lt_LT", "lv_LV", "my_MM", "ne_NP", "nl_XX", "ro_RO", "ru_RU", "si_LK", "tr_TR", "vi_VN", "zh_CN", "af_ZA", "az_AZ", "bn_IN", "fa_IR", "he_IL", "hr_HR", "id_ID", "ka_GE", "km_KH", "mk_MK", "ml_IN", "mn_MN", "mr_IN", "pl_PL", "ps_AF", "pt_XX", "sv_SE", "sw_KE", "ta_IN", "te_IN", "th_TH", "tl_XX", "uk_UA", "ur_PK", "xh_ZA", "gl_ES", "sl_SI"]
# {"allow-input":true}
# output_txt_path
translate_text(input_txt_path, source_lang, target_lang, max_length=512)

Using device: cuda
⚠️ Text too long; splitting into smaller chunks for translation.
Processing 7 chunks...


Translating chunks: 100%|██████████| 7/7 [00:16<00:00,  2.30s/chunk]


'Translation saved to /content/drive/MyDrive/mp3_to_md_whisper/_fr_XX_pl_PL_fr2'

In [ ]:
#@title reading txt fiel and translating from chinese to  other language{ display-mode: "form" }
# Data = "data_dst" #@param ["data_src", "data_dst"]
input_txt_path = "/content/drive/MyDrive/mp3_to_md_whisper/biomap-2025-03-18-08-23-15_transcription.txt" #@param [""] {allow-input: true}

# output_txt_path
translate_text(input_txt_path, source_lang='zh_CN', target_lang='en_XX', max_length=512)